<a href="https://colab.research.google.com/github/Richish/hands_on_ml/blob/master/8_Dimensionality_reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curse of dimensionality

Many Machine Learning problems involve thousands or even millions of features for each training instance. 
Not only does this make training extremely slow, it can also make it much harder to find a good solution. This problem is often referred to as the curse of dimensionality.

It turns out that many things behave very differently in high-dimensional space. For example, if you pick a random point in a unit square (a 1 × 1 square), it will have only about a 0.4% chance of being located less than 0.001 from a border (in other words, it is very unlikely that a random point will be “extreme” along any dimension). But in a 10,000-dimensional unit hypercube (a 1 × 1 × ⋯ × 1 cube, with ten thousand 1s), this probability is greater than 99.999999%. Most points in a high-dimensional hypercube are very close to the border.

More importantly:
If you pick two points randomly in a unit square, the distance between these two points will be, on average, roughly 0.52. If you pick two random points in a unit 3D cube, the average distance will be roughly 0.66. But what about two points picked randomly in a 1,000,000-dimensional hypercube? the average distance, will be about 408.25 (roughly sqrt(1,000,000/6!)) . This fact implies that high- dimensional datasets are at risk of being very sparse: most training instances are likely to be far away from each other. Of course, this also means that a new instance will likely be far away from any training instance, making predictions much less reliable than in lower dimensions, since they will be based on much larger extrapolations. In short, the more dimensions the training set has, the greater the risk of overfitting it.

With just 100 features (much less than in the MNIST problem), you would need more training instances than atoms in the observable universe in order for training instances to be within 0.1 of each other on average, assuming they were spread out uniformly across all dimensions.


Reducing dimensionality does lose some information (just like compressing an image to JPEG can degrade its quality), so even though it will speed up training, it may also make your system perform slightly worse. It also makes your pipelines a bit more complex and thus harder to maintain. So you should first try to train your system with the original data before considering using dimensionality reduction if training is too slow. In some cases, however, reducing the dimensionality of the training data may filter out some noise and unnecessary details and thus result in higher performance (but in general it won’t; it will just speed up training).


Apart from speeding up training, dimensionality reduction is also extremely useful for data visualization (or DataViz). Reducing the number of dimensions down to two (or three) makes it possible to plot a condensed view of a high-dimensional training set on a graph and often gain some important insights by visually detecting patterns, such as clusters. Moreover, DataViz is essential to communicate your conclusions to people who are not data scientists, in particular decision makers who will use your results.

# 2 main approaches for dimensionality reduction:


1.   Projection.
2.   Manifold learning.




## Projection

In most real-world problems, training instances are not spread out uniformly across all dimensions. Many features are almost constant, while others are highly correlated (as for MNIST). As a result, all training instances actually lie within (or close to) a much lower-dimensional subspace of the high-dimensional space. 

Example a 2-d subspace(plane) inside a 3-d space. obtained by projecting all points in 3-d to a 2-d space (may or maynot be parallel to any axes).

### Where projection cannot be used:
 In many cases the subspace may twist and turn, such as in the famous Swiss roll toy data‐set.
Simply projecting onto a plane (e.g., by dropping x3) would squash different layers of the Swiss roll together. However, what you really want is to unroll the Swiss roll to obtain the 2D dataset.

## Manifold learning

The Swiss roll is an example of a 2D manifold. Put simply, a 2D manifold is a 2D shape that can be bent and twisted in a higher-dimensional space. More generally, a d-dimensional manifold is a part of an n-dimensional space (where d < n) that locally resembles a d-dimensional hyperplane. In the case of the Swiss roll, d = 2 and n = 3: it locally resembles a 2D plane, but it is rolled in the third dimension.

Many dimensionality reduction algorithms work by modeling the manifold on which the training instances lie; this is called Manifold Learning. It relies on the manifold assumption, also called the manifold hypothesis, which holds that most real-world high-dimensional datasets lie close to a much lower-dimensional manifold. This assumption is very often empirically observed.

Once again, think about the MNIST dataset: all handwritten digit images have some similarities. They are made of connected lines, the borders are white, they are more or less centered, and so on. If you randomly generated images, only a ridiculously tiny fraction of them would look like handwritten digits. In other words, the degrees of freedom available to you if you try to create a digit image are dramatically lower than the degrees of freedom you would have if you were allowed to generate any image you wanted. These constraints tend to squeeze the dataset into a lower- dimensional manifold

The manifold assumption is often accompanied by another implicit assumption: that the task at hand (e.g., classification or regression) will be simpler if expressed in the lower-dimensional space of the manifold. However, this assumption does not always hold. 

In short, if you reduce the dimensionality of your training set before training a model, it will usually speed up training, but it may not always lead to a better or sim‐ pler solution; it all depends on the dataset.

# Dimensionality reduction algorithms:
1. PCA
2. 

## PCA
Principal Component Analysis (PCA) is the most popular dimensionality reduction algorithm. First it identifies the hyperplane that lies closest to the data, and then it projects the data onto it.




### Preserving the variance:
Before you can project the training set onto a lower-dimensional hyperplane, you first need to choose the right hyperplane.

Choose a hyperplane so that there is minimal reduction in variance (in data).
In other words, select the axis that preserves the maximum amount of var‐ iance, as it will most likely lose less information than the other projections. Another way to justify this choice is that it is the axis that minimizes the mean squared dis‐ tance between the original dataset and its projection onto that axis.

### Principal Components

PCA identifies the axis that accounts for the largest amount of variance in the training set. For 2-d data, It also finds a second axis, **orthogonal to the first one, that accounts for the largest amount of remaining variance**. If it were a higher-dimensional data‐ set, PCA would also find a third axis, orthogonal to both previous axes, and a fourth, a fifth, and so on—as many axes as the number of dimensions in the dataset.

The unit vector that defines the ith axis is called the ith principal component (PC). 

The direction of the principal components is not stable: if you perturb the training set slightly and run PCA again, some of the new PCs may point in the opposite direction of the original PCs. However, they will generally still lie on the same axes. In some cases, a pair of PCs may even rotate or swap, but the plane they define will generally remain the same.


### How to find principal components of a training set (Singular value decomposition)

There is a standard matrix factorization technique called Singular Value Decomposition (SVD) that can decompose the training set matrix X into the matrix multiplication of three matrices U Σ VT, where V contains all the principal components that we are looking for.

PCA assumes that the dataset is centered around the origin. As we will see, Scikit-Learn’s PCA classes take care of centering the data for you. However, if you implement PCA yourself (say using numpy), or if you use other libraries, don’t forget to center the data first.

#### svd using numpy only

In [ ]:
# Numpy provides a svd() method.
import numpy as np
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]


NameError: ignored

### Projecting down to d dimensions(from n dimensions):

Once you have identified all the principal components, you can reduce the dimensionality of the dataset down to d dimensions by projecting it onto the hyperplane defined by the first d principal components. Selecting this hyperplane ensures that the projection will preserve as much variance as possible. For example,the 3D dataset is projected down to the 2D plane defined by the first two principal components, preserving a large part of the dataset’s variance. As a result, the 2D projection looks very much like the original 3D dataset.

To project the training set onto the hyperplane, you can simply compute the matrix multiplication of the training set matrix X by the matrix Wd, defined as the matrix containing the first d principal components (i.e., the matrix composed of the first d columns of V). Projecting the training set down to d dimensions, Xd‐proj = X.Wd

The Python code in next cell projects the training set onto the plane defined by the first two principal components. You now know how to reduce the dimensionality of any dataset down to any number of dimensions, while preserving as much variance as possible.

In [ ]:
W2 = Vt.T[:, :2] 
X2D = X_centered.dot(W2)

NameError: ignored

### PCA with sk-learn

Scikit-Learn’s PCA class implements PCA using SVD decomposition just like we did before. The code in next cell applies PCA to reduce the dimensionality of the dataset down to two dimensions (note that it automatically takes care of centering the data):

In [ ]:
from sklearn.decomposition import PCA 
pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)

NameError: ignored

After fitting the PCA transformer to the dataset, you can access the principal components using the **components_** variable (note that it contains the PCs as horizontal vectors, so, for example, the first principal component is equal to pca.components_.T[:, 0]).

## Explained Variance Ratio
Very useful piece of information is the explained variance ratio of each principal component, available via the **explained_variance_ratio_** variable. It **indicates the proportion of the dataset’s variance that lies along the axis of each principal component**. 
For example, if the explained variance ratios of the first two components of the 3D dataset:
pca.explained_variance_ratio_ is array([0.84248607, 0.14631839])

This tells you that 84.2% of the dataset’s variance lies along the first axis, and 14.6% lies along the second axis. This leaves less than 1.2% for the third axis, so it is reason‐ able to assume that it probably carries little information.

## Choosing the Right Number of Dimensions
Instead of arbitrarily choosing the number of dimensions to reduce down to, it is generally preferable to choose the number of dimensions that add up to a sufficiently large portion of the variance (e.g., 95%). Unless, you are reducing dimensionality for data visualization—in that case you will generally want to reduce the dimensionality down to 2 or 3.

In [ ]:
# The following code computes PCA without reducing dimensionality, 
# then computes the minimum number of dimensions required to preserve 95% of the training set’s variance:
pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

NameError: ignored

In [ ]:
# You could then set n_components=d and run PCA again. 
# However, there is a much better option: 
# instead of specifying the number of principal components you want to preserve, 
# you can set n_components to be a float between 0.0 and 1.0, indicating the ratio of variance you wish to preserve:

pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

NameError: ignored

Yet another option is to plot the explained variance as a function of the number of dimensions (simply plot cumsum). There will usually be an elbow in the curve, where the explained variance stops growing fast. You can think of this as the intrinsic dimensionality of the dataset.

## PCA for compression

After dimensionality reduction, the training set takes up much less space. For example, try applying PCA to the MNIST dataset while preserving 95% of its variance. You should find that each instance will have just over 150 features, instead of the original 784 features. So while most of the variance is preserved, the dataset is now less than 20% of its original size! This is a reasonable compression ratio, and you can see how this can speed up a classification algorithm (such as an SVM classifier) tremendously.



    

In [1]:
from sklearn.datasets import fetch_openml
mnist=fetch_openml(name="mnist_784", version=1)
mnist.keys()



dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'DESCR', 'details', 'categories', 'url'])

In [2]:
# without PCA
from sklearn.model_selection import train_test_split
mnist['data'].shape
X,y=mnist['data'], mnist['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=60_000)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((60000, 784), (10000, 784), (60000,), (10000,))

In [3]:
# train on whole data - without pca
from sklearn.svm import SVC
svc = SVC()
svc.fit(X=X_train, y=y_train)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [4]:
# applying pca
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)
X_reduced.shape

(70000, 154)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, train_size=60_000)

In [6]:
# train using pca
from sklearn.svm import SVC
svc = SVC()
svc.fit(X=X_train, y=y_train)



SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [7]:
y_pred=svc.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_pred=y_pred, y_true=y_test)

0.9844

### Decompression:
It is also possible to decompress the reduced dataset back to 784 dimensions by applying the inverse transformation of the PCA projection. Of course this won’t give you back the original data, since the projection lost a bit of information (within the 5% variance that was dropped), but it will likely be quite close to the original data. The mean squared distance between the original data and the reconstructed data (compressed and then decompressed) is called the reconstruction error. For example, the following code compresses the MNIST dataset down to 154 dimensions, then uses the inverse_transform() method to decompress it back to 784 dimensions. 

In [8]:
pca = PCA(n_components = 154)
X_reduced = pca.fit_transform(X_train)
X_recovered = pca.inverse_transform(X_reduced)

Xrecovered = X{d‐proj}.W{dT}

## Randomized PCA.

Time complexity of PCA is O(m × n^2) + O(n^3) for the full SVD approach.

This time complexity can be reduced to O(m × d^2) + O(d^3) by using randomized PCA.

If you set the svd_solver hyperparameter to "randomized", Scikit-Learn uses a stochastic algorithm called Randomized PCA that quickly finds an approximation of the first d principal components.

It is dramatically faster than full SVD when d is much smaller than n.

By default, svd_solver is actually set to "auto": Scikit-Learn automatically uses the randomized PCA algorithm if m or n is greater than 500 and d is less than 80% of m or n, or else it uses the full SVD approach. If you want to force Scikit-Learn to use full SVD, you can set the svd_solver hyperparameter to "full".

In [9]:
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_train)

## Incremental PCA

One problem with the normal implementations of PCA is that they require the whole training set to fit in memory in order for the algorithm to run. Fortunately, Incremental PCA (IPCA) algorithms have been developed: you can split the training set into mini-batches and feed an I-PCA algorithm one mini-batch at a time. This is useful for large training sets, and also to apply PCA online.

The following code splits the MNIST dataset into 100 mini-batches (using NumPy’s array_split() function) and feeds them to Scikit-Learn’s IncrementalPCA class to reduce the dimensionality of the MNIST dataset down to 154 dimensions (just like before). Note that you must call the partial_fit() method with each mini-batch rather than the fit() method with the whole training set:

In [10]:
from sklearn.decomposition import IncrementalPCA
import numpy as np
n_batches=100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(ary=X_train, indices_or_sections=n_batches):
    inc_pca.fit(X=X_batch)
X_reduced = inc_pca.transform(X_train)
X_reduced.shape

(60000, 154)

### Using NumPy’s memmap

Alternatively, you can use NumPy’s memmap class, which allows you to manipulate a large array stored in a binary file on disk as if it were entirely in memory; the class loads only the data it needs in memory, when it needs it. 

Since the IncrementalPCA class uses only a small part of the array at any given time, the memory usage remains under control. This makes it possible to call the usual fit() method, as you can see in the following code:
    

In [11]:
X_mm = np.memmap(filename, dtype="float32", mode="readonly", shape=(m, n))
batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

NameError: ignored

## Kernel PCA

The kernel trick(same as in SVC) is a mathematical technique that implicitly maps instances into a very high-dimensional space (called the feature space), enabling nonlinear classification and regression with Support Vector Machines. Recall that a linear decision boundary in the high-dimensional feature space corresponds to a complex nonlinear decision boundary in the original space.

It turns out that the same trick can be applied to PCA, making it possible to perform complex nonlinear projections for dimensionality reduction. This is called Kernel PCA (kPCA).

It is often good at preserving clusters of instances after projection, or sometimes even unrolling datasets that lie close to a twisted manifold.
For example, the following code uses Scikit-Learn’s KernelPCA class to perform kPCA with an RBF kernel:


In [ ]:
from sklearn.decomposition import KernelPCA
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)
X_reduced.shape

### Selecting a kernel and hyperparameter tuning

As kPCA is an unsupervised learning algorithm, there is no obvious performance measure to help you select the best kernel and hyperparameter values. However, dimensionality reduction is often a preparation step for a supervised learning task (e.g., classification), so you can simply use grid search to select the kernel and hyperparameters that lead to the best performance on that task.

Example: the following code creates a two-step pipeline, first reducing dimensionality to two dimensions using kPCA, then applying Logistic Regression for classification. Then it uses Grid SearchCV to find the best kernel and gamma value for kPCA in order to get the best classification accuracy at the end of the pipeline:

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.decomposition import KernelPCA
import numpy as np

clf = Pipeline([
                ("kpca", KernelPCA(n_components=2)),
                ("log_svc", LogisticRegression())
])

param_grid = [{
    "kpca__gamma": np.linspace(0.03, 0.05, 10),
    "kpca__kernel": ["rbf", "sigmoid"]
}]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)

print(grid_search.best_params_)

Another approach, this time entirely unsupervised, is to select the kernel and hyper‐ parameters that yield the lowest reconstruction error. 

However, reconstruction is not as easy as with linear PCA. Becuase, the reconstructed point would lie in feature space, not in the original space. Since the feature space is infinite-dimensional, we cannot compute the reconstructed point, and therefore we cannot compute the true reconstruction error. Fortunately, it is possible to find a point in the original space that would map close to the reconstructed point. This is called the reconstruction pre-image. Once you have this pre-image, you can measure its squared distance to the original instance. You can then select the kernel and hyperparameters that minimize this reconstruction pre-image error.

It is done as follows in sklearn:
Notice: set fit_inverse_transform=True. 
By default, fit_inverse_transform=False and KernelPCA has no inverse_transform() method. This method only gets created when you set fit_inverse_transform=True.

In [ ]:
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.0433, 
                    fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)

In [ ]:
# compute the reconstruction pre-image error:
from sklearn.metrics import mean_squared_error
mean_squared_error(X, X_preimage)

# LLE (Local linear embedding) - one of manifold learning technoques.
 
It is a very powerful nonlinear dimensionality reduction (NLDR) technique. It is a Manifold Learning technique that does not rely on projections.

LLE works by first measur‐ ing how each training instance linearly relates to its closest neighbors (c.n.), and then looking for a low-dimensional representation of the training set where these local relationships are best preserved.
This makes it particularly good at unrolling twisted manifolds, especially when there is not too much noise.



In [ ]:
from sklearn.manifold import LocallyLinearEmbedding
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)

## How LLE works internally:

first, for each training instance x(i), the algorithm identifies its
k closest neighbors (in the preceding code k = 10), then tries to reconstruct x(i) as a linear function of these neighbors. More specifically, it finds the weights wi,j such that
the squared distance between x(i) and ∑m w x j is as small as possible, assuming w j=1 i,j i,j
= 0 if x(j) is not one of the k closest neighbors of x(i). 

Thus the first step of LLE is the constrained optimization problem, where W is the weight matrix containing all the weights wi,j. The second constraint simply normalizes the weights for each training instance x(i).

After this step, the weight matrix W (containing the weights wi, j) encodes the local linear relationships between the training instances. Now the second step is to map the
training instances into a d-dimensional space (where d < n) while preserving these
local relationships as much as possible. If z(i) is the image of x(i) in this d-dimensional
space, then we want the squared distance between z(i) and ∑m w z j   to be as small j=1 i,j
as possible. This idea leads to the unconstrained optimization problem described. It looks very similar to the first step, but instead of keeping the instances fixed and finding the optimal weights, we are doing the reverse: keeping the weights fixed and finding the optimal position of the instances’ images in the low- dimensional space. Note that Z is the matrix containing all z(i).



## Running time of LLE

O(m log(m)n log(k)) for finding the k nearest neighbors, O(mnk^3) for optimizing the weights, and O(dm^2) for constructing the low-dimensional representations. **The m^2 in the last term makes this algorithm scale poorly to very large datasets**.


# Other Dimensionality Reduction Techniques



##Multidimensional Scaling (MDS) 
reduces dimensionality while trying to preserve the distances between the instances

## Isomap 
creates a graph by connecting each instance to its nearest neighbors, then reduces dimensionality while trying to preserve the geodesic distances between the instances.

The geodesic distance between two nodes in a graph is the number of nodes on the shortest path between these nodes.

## t-Distributed Stochastic Neighbor Embedding (t-SNE) 
reduces dimensionality while trying to keep similar instances close and dissimilar instances apart. It is mostly used for visualization, in particular to visualize clusters of instances in high-dimensional space (e.g., to visualize the MNIST images in 2D).

## Linear Discriminant Analysis (LDA) 
It is actually a classification algorithm, but during training it learns the most discriminative axes between the classes, and these axes can then be used to define a hyperplane onto which to project the data. The benefit is that the projection will keep classes as far apart as possible, so LDA is a good technique to reduce dimensionality before running another classification algorithm such as an SVM classifier.